In [ ]:
使用awk从fastq文件中获取序列长度分布

zcat file.fastq.gz | awk 'NR%4 == 2 {lengths[length($0)]++} END {for (l in lengths) {print l, lengths[l]}}'  
反向互补序列(我使用的时候需要设计引物)

echo 'ATTGCTATGCTNNNT' | rev | tr 'ACTG' 'TGAC'
使用csplit将多个文件分割成单个文件：

csplit -z -q -n 4 -f sequence_ sequences.fasta /\>/ {*}  
通过awk将多FASTA文件分割成单个FASTA文件

awk '/^>/{s=++d".fa"} {print > s}' multi.fa
线性化多线性

cat file.fasta | awk '/^>/{if(N>0) printf("\n"); ++N; printf("%s\t",$0);next;} {printf("%s",$0);}END{printf("\n");}'
awk 'BEGIN{RS=">"}NR>1{sub("\n","\t"); gsub("\n",""); print RS$0}' file.fa
fastq2fasta

zcat file.fastq.gz | paste - - - - | perl -ane 'print ">$F[0]\n$F[2]\n";' | gzip -c > file.fasta.gz
bam2bed

samtools view file.bam | perl -F'\t' -ane '$strand=($F[1]&16)?"-":"+";$length=1;$tmp=$F[5];$tmp =~ s/(\d+)[MD]/$length+=$1/eg;print "$F[2]\t$F[3]\t".($F[3]+$length)."\t$F[0]\t0\t$strand\n";' > file.bed
bam2wig

samtools mpileup -BQ0 file.sorted.bam | perl -pe '($c, $start, undef, $depth) = split;if ($c ne $lastC || $start != $lastStart+1) {print "fixedStep chrom=$c start=$start step=1 span=1\n";}$_ = $depth."\n";($lastC, $lastStart) = ($c, $start);' | gzip -c > file.wig.gz
Number of reads in a fastq file

cat file.fq | echo $((`wc -l`/4))
Single line fasta file to multi-line fasta of 60 characteres each line

awk -v FS= '/^>/{print;next}{for (i=0;i<=NF/60;i++) {for (j=1;j<=60;j++) printf "%s", $(i*60 +j); print ""}}' file

fold -w 60 file
Sequence length of every entry in a multifasta file

awk '/^>/ {if (seqlen){print seqlen}; print ;seqlen=0;next; } { seqlen = seqlen +length($0)}END{print seqlen}' file.fa
Reproducible subsampling of a FASTQ file. srand() is the seed for the random number generator - keeps the subsampling the same when the script is run multiple times. 0.01 is the % of reads to output.

cat file.fq | paste - - - - | awk 'BEGIN{srand(1234)}{if(rand() < 0.01) print $0}' | tr '\t' '\n' > out.fq
or look at the Hengli's Seqtk

Deinterleaving a FASTQ:

cat file.fq | paste - - - - - - - - | tee >(cut -f1-4 | tr '\t'  
'\n' > out1.fq) | cut -f5-8 | tr '\t' '\n' > out2.fq
Using mpileup for a whole genome can take forever. So, handling each chromosome separately and parallely running them on several cores will speed up your pipeline. Using xargs you can easily realize it.

Example usage of xargs (-P is the number of parallel processes started - don't use more than the number of cores you have available):

samtools view -H yourFile.bam | grep "\@SQ" | sed 's/^.*SN://g' | cut -f 1 | xargs -I {} -n 1 -P 24 sh -c "samtools mpileup -BQ0 -d 100000 -uf yourGenome.fa -r {} yourFile.bam | bcftools view -vcg - > tmp.{}.vcf"
To merge the results afterwards, you might want to do something like this:

samtools view -H yourFile.bam | grep "\@SQ" | sed 's/^.*SN://g' | cut -f 1 | perl -ane 'system("cat tmp.$F[0].bcf >> yourFile.vcf");'
split large file by id/label/column

awk '{print >> $1; close($1)}' input_file
split a bed file by chromosome:

cat nexterarapidcapture_exome_targetedregions_v1.2.bed | sort -k1,1 -k2,2n | sed 's/^chr//' | awk '{close(f);f=$1}{print > f".bed"}'

#or
awk '{print $0 >> $1".bed"}' example.bed
sort vcf file with header

cat my.vcf | awk '$0~"^#" { print $0; next } { print $0 | "sort -k1,1V -k2,2n" }'
Rename a file, bash string manipulation

for file in *gz
do zcat $file > ${file/bed.gz/bed}
gnu sed print invisible characters

cat my_file | sed -n 'l'
cat -A
exit a dead ssh session

~.

copy large files, copy the from_dir directory inside the to_dir directory

rsync -av from_dir  to_dir

## copy every file inside the frm_dir to to_dir
rsync -av from_dir/ to_dir

##re-copy the files avoiding completed ones:

rsync -avhP /from/dir /to/dir
make directory using the current date

mkdir $(date +%F)
all the folders' size in the current folder (GNU du)

du -h --max-depth=1
this one is a bit different, try it and see the difference

du -ch

the total size of current directory

du -sh .

disk usage

df -h

the column names of the file, install csvkit https://csvkit.readthedocs.org/en/0.9.1/

csvcut -n

open top with human readable size in Mb, Gb. install htop for better visualization

top -M

how many memeory are used in Gb

free -mg

print out unique rows based on the first and second column

awk '!a[$1,$2]++' input_file

sort -u -k1,2 file It will sort based on unique first and second column

do not wrap the lines using less

less -S

pretty output

fold -w 60
cat file.txt | column -t | less -S
pass tab as delimiter http://unix.stackexchange.com/questions/46910/is-it-a-bug-for-join-with-t-t

-t $'\t'

awk with the first line printed always

awk ' NR ==1 || ($10 > 1 && $11 > 0 && $18 > 0.001)' input_file

delete blank lines with sed

sed /^$/d

delete the last line

sed $d

awk to join files based on several columns

my github repo

### select lines from a file based on columns in another file
## http://unix.stackexchange.com/questions/134829/compare-two-columns-of-different-files-and-print-if-it-matches
awk -F"\t" 'NR==FNR{a[$1$2$3]++;next};a[$1$2$3] > 0' file2 file1 

Finally learned about the !$ in unix: take the last thing (word) from the previous command.
echo hello, world; echo !$ gives 'world'

Create a script of the last executed command:
echo "!!" > foo.sh

Reuse all parameter of the previous command line:
!*

find bam in current folder (search recursively) and copy it to a new directory using 5 CPUs
find . -name "*bam" | xargs -P5 -I{} rsync -av {} dest_dir

ls -X will group files by extension.

loop through all the chromosomes

for i in {1..22} X Y 
do
  echo $i
done
for i in in {01..22} will expand to 01 02 ...

change every other newline to tab:

paste is used to concatenate corresponding lines from files: paste file1 file2 file3 .... If one of the "file" arguments is "-", then lines are read from standard input. If there are 2 "-" arguments, then paste takes 2 lines from stdin. And so on.

cat test.txt  
0    ATTTTATTNGAAATAGTAGTGGG
0    CTCCCAAAATACTAAAATTATAA
1    TTTTAGTTATTTANGAGGTTGAG
1    CNTAATCTTAACTCACTACAACC
2    TTATAATTTTAGTATTTTGGGAG
2    CATATTAACCAAACTAATCTTAA
3    GGTTAATATGGTGAAATTTAAT
3    ACCTCAACCTCNTAAATAACTAA

cat test.txt| paste - -                               
0    ATTTTATTNGAAATAGTAGTGGG    0    CTCCCAAAATACTAAAATTATAA
1    TTTTAGTTATTTANGAGGTTGAG    1    CNTAATCTTAACTCACTACAACC
2    TTATAATTTTAGTATTTTGGGAG    2    CATATTAACCAAACTAATCTTAA
3    GGTTAATATGGTGAAATTTAAT     3    ACCTCAACCTCNTAAATAACTAA
ORS: output record seperator in awk var=condition?condition_if_true:condition_if_false is the ternary operator.

cat test.txt| awk 'ORS=NR%2?"\t":"\n"'          

0    ATTTTATTNGAAATAGTAGTGGG    0    CTCCCAAAATACTAAAATTATAA
1    TTTTAGTTATTTANGAGGTTGAG    1    CNTAATCTTAACTCACTACAACC
2    TTATAATTTTAGTATTTTGGGAG    2    CATATTAACCAAACTAATCTTAA
3    GGTTAATATGGTGAAATTTAAT     3    ACCTCAACCTCNTAAATAACTAA
awk

We can also use the concept of a conditional operator in print statement of the form print CONDITION ? PRINT_IF_TRUE_TEXT : PRINT_IF_FALSE_TEXT. For example, in the code below, we identify sequences with lengths > 14:

cat data/test.tsv
blah_C1	ACTGTCTGTCACTGTGTTGTGATGTTGTGTGTG
blah_C2	ACTTTATATATT
blah_C3	ACTTATATATATATA
blah_C4	ACTTATATATATATA
blah_C5	ACTTTATATATT	

awk '{print (length($2)>14) ? $0">14" : $0"<=14";}' data/test.tsv
blah_C1	ACTGTCTGTCACTGTGTTGTGATGTTGTGTGTG>14
blah_C2	ACTTTATATATT<=14
blah_C3	ACTTATATATATATA>14
blah_C4	ACTTATATATATATA>14
blah_C5	ACTTTATATATT<=14

awk 'NR==3{print "";next}{printf $1"\t"}{print $1}' data/test.tsv
blah_C1	blah_C1
blah_C2	blah_C2

blah_C4	blah_C4
blah_C5	blah_C5
You can also use getline to load the contents of another file in addition to the one you are reading, for example, in the statement given below, the while loop will load each line from test.tsv into k until no more lines are to be read:

awk 'BEGIN{while((getline k <"data/test.tsv")>0) print "BEGIN:"k}{print}' data/test.tsv
BEGIN:blah_C1	ACTGTCTGTCACTGTGTTGTGATGTTGTGTGTG
BEGIN:blah_C2	ACTTTATATATT
BEGIN:blah_C3	ACTTATATATATATA
BEGIN:blah_C4	ACTTATATATATATA
BEGIN:blah_C5	ACTTTATATATT
blah_C1	ACTGTCTGTCACTGTGTTGTGATGTTGTGTGTG
blah_C2	ACTTTATATATT
blah_C3	ACTTATATATATATA
blah_C4	ACTTATATATATATA
blah_C5	ACTTTATATATT
merge multiple fasta sequences in two files into a single file line by line

see post

linearize.awk:

/^>/ {printf("%s%s\t",(N>0?"\n":""),$0);N++;next;} {printf("%s",$0);} END {printf("\n");}
paste <(awk -f linearize.awk file1.fa ) <(awk -f linearize.awk file2.fa  )| tr "\t" "\n"
grep fastq reads containing a pattern but maintain the fastq format

grep -A 2 -B 1 'AAGTTGATAACGGACTAGCCTTATTTT' file.fq | sed '/^--$/d' > out.fq

# or
zcat reads.fq.gz \
| paste - - - - \
| awk -v FS="\t" -v OFS="\n" '$2 ~ "AAGTTGATAACGGACTAGCCTTATTTT" {print $1, $2, $3, $4}' \
| gzip > filtered.fq.gz
count how many columns of a tsv files:

cat file.tsv | head -1 | tr "\t" "\n" | wc -l  
csvcut -n -t  file.tsv (from csvkit)
awk '{print NF; exit}' file.tsv
awk -F "\t" 'NR == 1 {print NF}' file.tsv
combine info to the fasta header

from biostar post

cat myfasta.txt 
>Blap_contig79
MSTDVDAKTRSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI
>Bluc_contig23663
MSTNVDAKARSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI
>Blap_contig7988
MSTDVDAKTRSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI
>Bluc_contig1223663
MSTNVDAKARSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI

cat my_info.txt 
info1
info2
info3
info4

paste <(cat my_info.txt) <(cat myfasta.txt| paste - - | cut -c2-) | awk '{printf(">%s_%s\n%s\n",$1,$2,$3);}'
>info1_Blap_contig79
MSTDVDAKTRSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI
>info2_Bluc_contig23663
MSTNVDAKARSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI
>info3_Blap_contig7988
MSTDVDAKTRSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI
>info4_Bluc_contig1223663
MSTNVDAKARSKERASIAAFYVGRNIFVTGGTGFLGKVLIEKLLRSCPDVGEIFILMRPKAGLSI
count how many columns in a tsv file

cat file.tsv | head -1 | tr "\t" "\n" | wc -l  

##(from csvkit)
csvcut -n -t file.

## emulate csvcut -n -t
less files.tsv | head -1| tr "\t" "\n" | nl

awk -F "\t" 'NR == 1 {print NF}' file.tsv
awk '{print NF; exit}'
change fasta header

see https://www.biostars.org/p/53212/

The fasta header is like >7 dna:chromosome chromosome:GRCh37:7:1:159138663:1 convert to >7:

cat Homo_sapiens_assembly19.fasta | gawk '/^>/ { b=gensub(" dna:.+", "", "g", $0); print b; next} {print}' > Homo_sapiens_assembly19_reheader.fasta
mkdir and cd into that dir shortcut

mkdir blah && cd $_
cut out columns based on column names in another file

http://crazyhottommy.blogspot.com/2016/10/cutting-out-500-columns-from-26g-file.html

#! /bin/bash

set -e
set -u
set -o pipefail

#### Author: Ming Tang (Tommy)
#### Date 09/29/2016
#### I got the idea from this stackOverflow post http://stackoverflow.com/questions/11098189/awk-extract-columns-from-file-based-on-header-selected-from-2nd-file

# show help
show_help(){
cat << EOF
  This is a wrapper extracting columns of a (big) dataframe based on a list of column names in another
  file. The column names must be one per line. The output will be stdout. For small files < 2G, one 
  can load it into R and do it easily, but when the file is big > 10G. R is quite cubersome. 
  Using unix commands on the other hand is better because files do not have to be loaded into memory at once.
  e.g. subset a 26G size file for 700 columns takes around 30 mins. Memory footage is very low ~4MB.

  usage: ${0##*/} -f < a dataframe  > -c < colNames> -d <delimiter of the file>
        -h display this help and exit.
		-f the file you want to extract columns from. must contain a header with column names.
		-c a file with the one column name per line.
		-d delimiter of the dataframe: , or \t. default is tab.  
		
		e.g. 
		
		for tsv file:
			${0##*/} -f mydata.tsv -c colnames.txt -d $'\t' or simply ommit the -d, default is tab.
		
		for csv file: Note you have to specify -d , if your file is csv, otherwise all columns will be cut out.
			${0##*/} -f mydata.csv -c colnames.txt -d ,
        
EOF
}

## if there are no arguments provided, show help
if [[ $# == 0 ]]; then show_help; exit 1; fi

while getopts ":hf:c:d:" opt; do
  case "$opt" in
    h) show_help;exit 0;;
    f) File2extract=$OPTARG;;
    c) colNames=$OPTARG;;
    d) delim=$OPTARG;;
    '?') echo "Invalid option $OPTARG"; show_help >&2; exit 1;;
  esac
done
	

## set up the default delimiter to be tab, Note the way I specify tab 

delim=${delim:-$'\t'}

## get the number of columns in the data frame that match the column names in the colNames file.
## change the output to 2,5,6,22,... and get rid of the last comma  so cut -f can be used
 
cols=$(head -1 "${File2extract}" | tr "${delim}" "\n" | grep -nf "${colNames}" | sed 's/:.*$//' | tr "\n" "," | sed 's/,$//')

## cut out the columns 
cut -d"${delim}" -f"${cols}" "${File2extract}"
or use csvtk from Shen Wei:

csvtk cut -t -f $(paste -s -d , list.txt) data.tsv
merge all bed files and add a column for the filename.

awk '{print $0 "\t" FILENAME}' *bed 
add or remove chr from the start of each line

# add chr
sed 's/^/chr/' my.bed

# or
awk 'BEGIN {OFS = "\t"} {$1="chr"$1; print}'

# remove chr
sed 's/^chr//' my.bed
check if a tsv files have the same number of columns for all rows

awk '{print NF}' test.tsv | sort -nu | head -n 1
Parallelized samtools mpileup

https://www.biostars.org/p/134331/

BAM="yourFile.bam"
REF="reference.fasta"
samtools view -H $BAM | grep "\@SQ" | sed 's/^.*SN://g' | cut -f 1 | xargs -I {} -n 1 -P 24 sh -c "samtools mpileup -BQ0 -d 100000 -uf $REF -r \"{}\" $BAM | bcftools call -cv > \"{}\".vcf"
convert multiple lines to a single line

This is better than tr "\n" "\t" because somtimes I do not want to convert the last newline to tab.

cat myfile.txt | paste -s 
merge multiple files with same header by keeping the header of the first file

I usually do it in R, but like the quick solution.

https://stackoverflow.com/questions/16890582/unixmerge-multiple-csv-files-with-same-header-by-keeping-the-header-of-the-firs

awk 'FNR==1 && NR!=1{next;}{print}' *.csv 

# or

awk '
    FNR==1 && NR!=1 { while (/^<header>/) getline; }
    1 {print}
' file*.txt >all.txt
insert a field into the first line

cut -f1-4 F5.hg38.enhancers.expression.usage.matrix | head
CNhs11844	CNhs11251	CNhs11282	CNhs10746
chr10:100006233-100006603	1	0	0
chr10:100008181-100008444	0	0	0
chr10:100014348-100014634	0	0	0
chr10:100020065-100020562	0	0	0
chr10:100043485-100043744	0	0	0
chr10:100114218-100114567	0	0	0
chr10:100148595-100148922	0	0	0
chr10:100182422-100182522	0	0	0
chr10:100184498-100184704	0	0	0

sed '1 s/^/enhancer\t/' F5.hg38.enhancers.expression.usage.matrix | cut -f1-4 | head
enhancer	CNhs11844	CNhs11251	CNhs11282
chr10:100006233-100006603	1	0	0
chr10:100008181-100008444	0	0	0
chr10:100014348-100014634	0	0	0
chr10:100020065-100020562	0	0	0
chr10:100043485-100043744	0	0	0
chr10:100114218-100114567	0	0	0
chr10:100148595-100148922	0	0	0
chr10:100182422-100182522	0	0	0
chr10:100184498-100184704	0	0	0

In [ ]:
Part I: Working With Files

1. Empty a file (truncate to 0 size)

$ > file

This one-liner uses the output redirection operator >. Redirection of output causes the file to be opened for writing. If the file does not exist it is created; if it does exist it is truncated to zero size. As we're not redirecting anything to the file it remains empty.

If you wish to replace the contents of a file with some string or create a file with specific content, you can do this:

$ echo "some string" > file

This puts the string "some string" in the file.

2. Append a string to a file

$ echo "foo bar baz" >> file

This one-liner uses a different output redirection operator >>, which appends to the file. If the file does not exist it is created. The string appended to the file is followed by a newline. If you don't want a newline appended after the string, add the -n argument to echo:

$ echo -n "foo bar baz" >> file

3. Read the first line from a file and put it in a variable

$ read -r line < file

This one-liner uses the built-in bash command read and the input redirection operator <. The read command reads one line from the standard input and puts it in the line variable. The -r parameter makes sure the input is read raw, meaning the backslashes won't get escaped (they'll be left as is). The redirection command < file opens file for reading and makes it the standard input to the read command.

The read command removes all characters present in the special IFS variable. IFS stands for Internal Field Separator that is used for word splitting after expansion and to split lines into words with the read built-in command. By default IFS contains space, tab, and newline, which means that the leading and trailing tabs and spaces will get removed. If you wish to preserve them, you can set IFS to nothing for the time being:

$ IFS= read -r line < file

This will change the value of IFS just for this command and will make sure the first line gets read into the line variable really raw with all the leading and trailing whitespaces.

Another way to read the first line from a file into a variable is to do this:

$ line=$(head -1 file)

This one-liner uses the command substitution operator $(...). It runs the command in ..., and returns its output. In this case the command is head -1 file that outputs the first line of the file. The output is then assigned to the line variable. Using $(...) is exactly the same as `...`, so you could have also written:

$ line=`head -1 file`

However $(...) is the preferred way in bash as it's cleaner and easier to nest.

4. Read a file line-by-line

$ while read -r line; do
    # do something with $line
done < file

This is the one and only right way to read lines from a file one-by-one. This method puts the read command in a while loop. When the read command encounters end-of-file, it returns a positive return code (code for failure) and the while loop stops.

Remember that read trims leading and trailing whitespace, so if you wish to preserve it, clear the IFS variable:

$ while IFS= read -r line; do
    # do something with $line
done < file

If you don't like the to put < file at the end, you can also pipe the contents of the file to the while loop:

$ cat file | while IFS= read -r line; do
    # do something with $line
done

5. Read a random line from a file and put it in a variable

$ read -r random_line < <(shuf file)

There is no clean way to read a random line from a file with just bash, so we'll need to use some external programs for help. If you're on a modern Linux machine, then it comes with the shuf utility that's in GNU coreutils.

This one-liner uses the process substitution <(...) operator. This process substitution operator creates an anonymous named pipe, and connects the stdout of the process to the write part of the named pipe. Then bash executes the process, and it replaces the whole process substitution expression with the filename of the anonymous named pipe.

When bash sees <(shuf file) it opens a special file /dev/fd/n, where n is a free file descriptor, then runs shuf file with its stdout connected to /dev/fd/n and replaces <(shuf file) with /dev/fd/n so the command effectively becomes:

$ read -r random_line < /dev/fd/n

Which reads the first line from the shuffled file.

Here is another way to do it with the help of GNU sort. GNU sort takes the -R option that randomizes the input.

$ read -r random_line < <(sort -R file)

Another way to get a random line in a variable is this:

$ random_line=$(sort -R file | head -1)

Here the file gets randomly sorted by sort -R and then head -1 takes the first line.

6. Read the first three columns/fields from a file into variables

$ while read -r field1 field2 field3 throwaway; do
    # do something with $field1, $field2, and $field3
done < file

If you specify more than one variable name to the read command, it shall split the line into fields (splitting is done based on what's in the IFS variable, which contains a whitespace, a tab, and a newline by default), and put the first field in the first variable, the second field in the second variable, etc., and it will put the remaining fields in the last variable. That's why we have the throwaway variable after the three field variables. if we didn't have it, and the file had more than three columns, the third field would also get the leftovers.

Sometimes it's shorter to just write _ for the throwaway variable:

$ while read -r field1 field2 field3 _; do
    # do something with $field1, $field2, and $field3
done < file

Or if you have a file with exactly three fields, then you don't need it at all:

$ while read -r field1 field2 field3; do
    # do something with $field1, $field2, and $field3
done < file

Here is an example. Let's say you wish to find out number of lines, number of words, and number of bytes in a file. If you run wc on a file you get these 3 numbers plus the filename as the fourth field:

$ cat file-with-5-lines
x 1
x 2
x 3
x 4
x 5

$ wc file-with-5-lines
 5 10 20 file-with-5-lines

So this file has 5 lines, 10 words, and 20 chars. We can use the read command to get this info into variables:

$ read lines words chars _ < <(wc file-with-5-lines)

$ echo $lines
5
$ echo $words
10
$ echo $chars
20

Similarly you can use here-strings to split strings into variables. Let's say you have a string "20 packets in 10 seconds" in a $info variable and you want to extract 20 and 10. Not too long ago I'd have written this:

$ packets=$(echo $info | awk '{ print $1 }')
$ time=$(echo $info | awk '{ print $4 }')

However given the power of read and our bash knowledge, we can now do this:

$ read packets _ _ time _ <<< "$info"

Here the <<< is a here-string, which lets you pass strings directly to the standard input of commands.

7. Find the size of a file, and put it in a variable

$ size=$(wc -c < file)

This one-liner uses the command substitution operator $(...) that I explained in one-liner #3. It runs the command in ..., and returns its output. In this case the command is wc -c < file that prints the number of chars (bytes) in the file. The output is then assigned to size variable.

8. Extract the filename from the path

Let's say you have a /path/to/file.ext, and you wish to extract just the filename file.ext. How do you do it? A good solution is to use the parameter expansion mechanism:

$ filename=${path##*/}

This one-liner uses the ${var##pattern} parameter expansion. This expansion tries to match the pattern at the beginning of the $var variable. If it matches, then the result of the expansion is the value of $var with the longest matching pattern deleted.

In this case the pattern is */ which matches at the beginning of /path/to/file.ext and as it's a greedy match, the pattern matches all the way till the last slash (it matches /path/to/). The result of this expansion is then just the filename file.ext as the matched pattern gets deleted.

9. Extract the directory name from the path

This is similar to the previous one-liner. Let's say you have a /path/to/file.ext, and you wish to extract just the path to the file /path/to. You can use the parameter expansion again:

$ dirname=${path%/*}

This time it's the ${var%pattern} parameter expansion that tries to match the pattern at the end of the $var variable. If the pattern matches, then the result of the expansion is the value of $var shortest matching pattern deleted.

In this case the pattern is /*, which matches at the end of /path/to/file.ext (it matches /file.ext). The result then is just the dirname /path/to as the matched pattern gets deleted.

10. Make a copy of a file quickly

Let's say you wish to copy the file at /path/to/file to /path/to/file_copy. Normally you'd write:

$ cp /path/to/file /path/to/file_copy

However you can do it much quicker by using the brace expansion {...}:

$ cp /path/to/file{,_copy}

Brace expansion is a mechanism by which arbitrary strings can be generated. In this particular case /path/to/file{,_copy} generates the string /path/to/file /path/to/file_copy, and the whole command becomes cp /path/to/file /path/to/file_copy.

Similarly you can move a file quickly:

$ mv /path/to/file{,_old}

This expands to mv /path/to/file /path/to/file_old.

In [ ]:
rt II: Working With Strings

1. Generate the alphabet from a-z

$ echo {a..z}

This one-liner uses brace expansion. Brace expansion is a mechanism for generating arbitrary strings. This one-liner uses a sequence expression of the form {x..y}, where x and y are single characters. The sequence expression expands to each character lexicographically between x and y, inclusive.

If you run it, you get all the letters from a-z:

$ echo {a..z}
a b c d e f g h i j k l m n o p q r s t u v w x y z

2. Generate the alphabet from a-z without spaces between characters

$ printf "%c" {a..z}

This is an awesome bash trick that 99.99% bash users don't know about. If you supply a list of items to the printf function it actually applies the format in a loop until the list is empty! printf as a loop! There is nothing more awesome than that!

In this one-liner the printf format is "%c", which means "a character" and the arguments are all letters from a-z separated by space. So what printf does is it iterates over the list outputting each character after character until it runs out of letters.

Here is the output if you run it:

abcdefghijklmnopqrstuvwxyz

This output is without a terminating newline because the format string was "%c" and it doesn't include \n. To have it newline terminated, just add $'\n' to the list of chars to print:

$ printf "%c" {a..z} $'\n'

$'\n' is bash idiomatic way to represent a newline character. printf then just prints chars a to z, and the newline character.

Another way to add a trailing newline character is to echo the output of printf:

$ echo $(printf "%c" {a..z})

This one-liner uses command substitution, which runs printf "%c" {a..z} and replaces the command with its output. Then echo prints this output and adds a newline itself.

Want to output all letters in a column instead? Add a newline after each character!

$ printf "%c\n" {a..z}

Output:

a
b
...
z

Want to put the output from printf in a variable quickly? Use the -v argument:

$ printf -v alphabet "%c" {a..z}

This puts abcdefghijklmnopqrstuvwxyz in the $alphabet variable.

Similarly you can generate a list of numbers. Let's say from 1 to 100:

$ echo {1..100}

Output:

1 2 3 ... 100

Alternatively, if you forget this method, you can use the external seq utility to generate a sequence of numbers:

$ seq 1 100

3. Pad numbers 0 to 9 with a leading zero

$ printf "%02d " {0..9}

Here we use the looping abilities of printf again. This time the format is "%02d ", which means "zero pad the integer up to two positions", and the items to loop through are the numbers 0-9, generated by the brace expansion (as explained in the previous one-liner).

Output:

00 01 02 03 04 05 06 07 08 09

If you use bash 4, you can do the same with the new feature of brace expansion:

$ echo {00..09}

Older bashes don't have this feature.

4. Produce 30 English words

$ echo {w,t,}h{e{n{,ce{,forth}},re{,in,fore,with{,al}}},ither,at}

This is an abuse of brace expansion. Just look at what this produces:

when whence whenceforth where wherein wherefore wherewith wherewithal whither what then thence thenceforth there therein therefore therewith therewithal thither that hen hence henceforth here herein herefore herewith herewithal hither hat
Crazy awesome!

Here is how it works - you can produce permutations of words/symbols with brace expansion. For example, if you do this,

$ echo {a,b,c}{1,2,3}

It will produce the result a1 a2 a3 b1 b2 b3 c1 c2 c3. It takes the first a, and combines it with {1,2,3}, producing a1 a2 a3. Then it takes b and combines it with {1,2,3}, and then it does the same for c.

So this one-liner is just a smart combination of braces that when expanded produce all these English words!

5. Produce 10 copies of the same string

$ echo foo{,,,,,,,,,,}

This one-liner uses the brace expansion again. What happens here is foo gets combined with 10 empty strings, so the output is 10 copies of foo:

foo foo foo foo foo foo foo foo foo foo foo
6. Join two strings

$ echo "$x$y"

This one-liner simply concatenates two variables together. If the variable x contains foo and y contains bar then the result is foobar.

Notice that "$x$y" were quoted. If we didn't quote it, echo would interpret the $x$y as regular arguments, and would first try to parse them to see if they contain command line switches. So if $x contains something beginning with -, it would be a command line argument rather than an argument to echo:

x=-n
y=" foo"
echo $x$y
Output:

foo
Versus the correct way:

x=-n
y=" foo"
echo "$x$y"
Output:

-n foo
If you need to put the two joined strings in a variable, you can omit the quotes:

var=$x$y
7. Split a string on a given character

Let's say you have a string foo-bar-baz in the variable $str and you wish to split it on the dash and iterate over it. You can simply combine IFS with read to do it:

$ IFS=- read -r x y z <<< "$str"

Here we use the read x command that reads data from stdin and puts the data in the x y z variables. We set IFS to - as this variable is used for field splitting. If multiple variable names are specified to read, IFS is used to split the line of input so that each variable gets a single field of the input.

In this one-liner $x gets foo, $y gets bar, $z gets baz.

Also notice the use of <<< operator. This is the here-string operator that allows strings to be passed to stdin of commands easily. In this case string $str is passed as stdin to read.

You can also put the split fields and put them in an array:

$ IFS=- read -ra parts <<< "foo-bar-baz"

The -a argument to read makes it put the split words in the given array. In this case the array is parts. You can access array elements through ${parts[0]}, ${parts[1]}, and ${parts[0]}. Or just access all of them through ${parts[@]}.

8. Process a string character by character

$ while IFS= read -rn1 c; do
    # do something with $c
done <<< "$str"

Here we use the -n1 argument to read command to make it read the input character at a time. Similarly we can use -n2 to read two chars at a time, etc.

9. Replace "foo" with "bar" in a string

$ echo ${str/foo/bar}

This one-liner uses parameter expansion of form ${var/find/replace}. It finds the string find in var and replaces it with replace. Really simple!

To replace all occurrences of "foo" with "bar", use the ${var//find/replace} form:

$ echo ${str//foo/bar}

10. Check if a string matches a pattern

$ if [[ $file = *.zip ]]; then
    # do something
fi

Here the one-liner does something if $file matches *.zip. This is a simple glob pattern matching, and you can use symbols * ? [...] to do matching. Code * matches any string, ? matches a single char, and [...] matches any character in ... or a character class.

Here is another example that matches if answer is Y or y:

$ if [[ $answer = [Yy]* ]]; then
    # do something
fi

11. Check if a string matches a regular expression

$ if [[ $str =~ [0-9]+\.[0-9]+ ]]; then
    # do something
fi

This one-liner tests if the string $str matches regex [0-9]+\.[0-9]+, which means match a number followed by a dot followed by number. The format for regular expressions is described in man 3 regex.

12. Find the length of the string

$ echo ${#str}

Here we use parameter expansion ${#str} which returns the length of the string in variable str. Really simple.

13. Extract a substring from a string

$ str="hello world"
$ echo ${str:6}

This one-liner extracts world from hello world. It uses the substring expansion. In general substring expansion looks like ${var:offset:length}, and it extracts length characters from var starting at index offset. In our one-liner we omit the length that makes it extract all characters starting at offset 6.

Here is another example:

$ echo ${str:7:2}

Output:

or
14. Uppercase a string

$ declare -u var
$ var="foo bar"

The declare command in bash declares variables and/or gives them attributes. In this case we give the variable var attribute -u, which upper-cases its content whenever it gets assigned something. Now if you echo it, the contents will be upper-cased:

$ echo $var

FOO BAR
Note that -u argument was introduced in bash 4. Similarly you can use another feature of bash 4, which is the ${var^^} parameter expansion that upper-cases a string in var:

$ str="zoo raw"
$ echo ${str^^}

Output:

ZOO RAW
15. Lowercase a string

$ declare -l var
$ var="FOO BAR"

Similar to the previous one-liner, -l argument to declare sets the lower-case attribute on var, which makes it always be lower-case:

$ echo $var

foo bar
The -l argument is also available only in bash 4 and later.

Another way to lowercase a string is to use ${var,,} parameter expansion:

$ str="ZOO RAW"
$ echo ${str,,}

Output:

zoo raw

In [ ]:
Part IV: Working with history

1. Erase all shell history

$ rm ~/.bash_history

Bash keeps the shell history in a hidden file called .bash_history. This file is located in your home directory. To get rid of the history, just delete it.

Note that if you logout after erasing the shell history, this last rm ~/.bash_history command will be logged. If you want to hide that you erased shell history, see the next one-liner.

2. Stop logging history for this session

$ unset HISTFILE

The HISTFILE special bash variable points to the file where the shell history should be saved. If you unset it, bash won't save the history.

Alternatively you can point it to /dev/null,

$ HISTFILE=/dev/null

3. Don't log the current command to history

Just start the command with an extra space:

$  command

If the command starts with an extra space, it's not logged to history.

Note that this only works if the HISTIGNORE variable is properly configured. This variable contains : separated values of command prefixes that shouldn't be logged.

For example to ignore spaces set it to this:

HISTIGNORE="[ ]*"
My HISTIGNORE looks like this:

HISTIGNORE="&:[ ]*"
The ampersand has a special meaning - don't log repeated commands.

4. Change the file where bash logs command history

$ HISTFILE=~/docs/shell_history.txt

Here we simply change the HISTFILE special bash variable and point it to ~/docs/shell_history.txt. From now on bash will save the command history in that file.

5. Add timestamps to history log

$ HISTTIMEFORMAT="%Y-%m-%d %H:%M:%S"

If you set the HISTTIMEFORMAT special bash variable to a valid date format (see man 3 strftime) then bash will log the timestamps to the history log. It will also display them when you call the history command (see the next one-liner).

6. Show the history

$ history

The history command displays the history list with line numbers. If HISTTIMEFORMAT is set, it also displays the timestamps.

7. Show the last 50 commands from the history

$ history 50

If you specify a numeric argument, such as 50, to history, it prints the last 50 commands from the history.

7. Show the top 10 most used commands from the bash history

$ history |
    sed 's/^ \+//;s/  / /' |
    cut -d' ' -f2- |
    awk '{ count[$0]++ } END { for (i in count) print count[i], i }' |
    sort -rn |
    head -10

This one-liner combines bash with sed, cut, awk, sort and head. The perfect combination. Let's walk through this to understand what happens. Let's say the output of history is:

$ history
    1  rm .bash_history 
    2  dmesg
    3  su -
    4  man cryptsetup
    5  dmesg

First we use the sed command to remove the leading spaces and convert the double space after the history command number to a single space:

$ history | sed 's/^ \+//;s/  / /'
1 rm .bash_history 
2 dmesg
3 su -
4 man cryptsetup
5 dmesg

Next we use cut to remove the first column (the history numbers):

$ history |
    sed 's/^ \+//;s/  / /' |
    cut -d' ' -f2-

rm .bash_history 
dmesg
su -
man cryptsetup
dmesg

Next we use awk to record how many times each command has been seen:

$ history |
    sed 's/^ \+//;s/  / /' |
    cut -d' ' -f2- |
    awk '{ count[$0]++ } END { for (i in count) print count[i], i }'

1 rm .bash_history 
2 dmesg
1 su -
1 man cryptsetup

Then we sort the output numerically and reverse it:

$ history |
    sed 's/^ \+//;s/  / /' |
    cut -d' ' -f2- |
    awk '{ count[$0]++ } END { for (i in count) print count[i], i }' |
    sort -rn

2 dmesg
1 rm .bash_history 
1 su -
1 man cryptsetup

Finally we take the first 10 lines that correspond to 10 most frequently used commands:

$ history |
    sed 's/^ \+//;s/  / /' |
    cut -d' ' -f2- |
    awk '{ count[$0]++ } END { for (i in count) print count[i], i }' |
    sort -rn |
    head -10

Here is what my 10 most frequently used commands look like:

2172 ls
1610 gs
252 cd ..
215 gp
213 ls -las
197 cd projects
155 gpu
151 cd
119 gl
119 cd tests/

Here I've gs that's an alias for git status, gp is git push, gpu is git pull and gl is git log.

8. Execute the previous command quickly

$ !!

That's right. Type two bangs. The first bang starts history substitution, and the second one refers to the last command. Here is an example:

$ echo foo
foo
$ !!
foo

Here the echo foo command was repeated.

It's especially useful if you wanted to execute a command through sudo but forgot. Then all you've to do is run:

$ rm /var/log/something
rm: cannot remove `/var/log/something': Permission denied
$
$ sudo !!   # executes `sudo rm /var/log/something`

9. Execute the most recent command starting with the given string

$ !foo

The first bang starts history substitution, and the second one refers to the most recent command starting with foo.

For example,

$ echo foo
foo
$ ls /
/bin /boot /home /dev /proc /root /tmp
$ awk -F: '{print $2}' /etc/passwd
...
$ !ls
/bin /boot /home /dev /proc /root /tmp

Here we executed commands echo, ls, awk, and then used !ls to refer to the ls / command.

10. Open the previous command you executed in a text editor

$ fc

Fc opens the previous command in a text editor. It's useful if you've a longer, more complex command and want to edit it.

For example, let's say you've written a one-liner that has an error, such as:

$ for wav in wav/*; do mp3=$(sed 's/\.wav/\.mp3/' <<< "$wav"); ffmpeg -i "$wav" "$m3p"; done

And you can't see what's going on because you've to scroll around, then you can simply type fc to load it in your text editor, and then quickly find that you mistyped mp3 at the end.

In [ ]:
#ytube-dl.sh下载脚本
#使用  ./ytube-dl.sh "https://www.youtube.com/网址"
#!/usr/bin/env bash
video_url="${1}"
out_file_name="${2:-out.video}"
curl \
  -H 'Upgrade-insecure-requests: 1' \
  -H 'Cache-control: max-age=0' \
  -H 'Accept-language: en-US,en;q=0.8,bn;q=0.6' \
  -H 'Accept-encoding: gzip, deflate, sdch' \
  -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36' \
  "${video_url}" \
| gunzip \
| egrep -o 'https%3A%2F%2F[^\.]*\.googlevideo.com%2F[^,\]*' \
| perl -pe 's/\%(\w\w)/chr hex $1/ge' \
| head -n1 \
| xargs wget -O "${out_file_name}"


In [ ]:
http://www.bashoneliners.com/
用来搜索一行命令
https://github.com/adrian011494/Bash-One-Liners
python liners 命令名


In [ ]:
#replace the necesssary fields
ldapsearch -v -x -H ldap://10.0.0.39 -D CN=106114104,OU=2014,OU=UG,OU=CSE,DC=octa,DC=edu -W -b dc=octa,dc=edu -s sub "CN=106114104"

cat $path |cut -d ' ' -f 1|grep -v '^$'|sort|uniq -c

smbclient //octa.edu/A4-4515X <octa-pass> -U <Roll-No> -W octa.edu -I 10.0.0.38 -c "print <filename>; exit;"

seq 106114001 106114105|xargs -I {} sshpass -p {} ssh -o StrictHostKeyChecking=no {}@10.0.0.84 "chmod 755 ~{}"

sudo mount -t cifs //nas/<ug or pg>/<dept>/<roll no> <path_to_local_directory> -o username=<roll no>,workgroup=octa.edu
# provide octa password on prompt

---------------
subl
gnome-terminal --tab -e "bash" --working-directory=~/Desktop/Coding --tab -e "bash"
google-chrome --incognito &
google-chrome google.com youtube.com facebook.com &
gnome-open .
vlc /media/venkkatesh/Games\ and\ Movies/Songs/Bill/*.mp3 

------------------

In [ ]:
top -b -n1 -u test 'Show Processes and memory for a user'
find /proc/sys -type f | for line in $(cat -);do echo $line;cat $line;done  show all kernel parameter values
awk /Hug.*Tot/{print } /proc/meminfo  Number of hugepages
awk /Hug.*siz/{print } /proc/meminfo  Size of hugepages
awk /Hug.*Fre/{print } /proc/meminfo  Available of hugepages
ps -ef | awk NR { print } | sort -u   | wc -l  Count the unique users on system
ps -ef | awk NR { print } | sort -u   | uniq -c Show how many processes each logon is running

In [ ]:
# List all changed, added or deleted files.

git status -s | awk '{ print $2 }'

---------------
# List git commits made on Monday.

git log --pretty=format:"%h%x09%an%x09%ad%x09%s" | awk '$3 == "Mon"'
    
------------
# Kill whatever process is running on the given port.

if [ "$#" -ne 1 ]; then
  echo 'Usage: kill-port PORT'
  exit 1
fi

lsof -i:$1 | awk 'NR > 1 { print $2 }' | xargs kill -9
    
---------------
# Concatenate all js files
# and separate by line breaks.

find . -name '*.js' | xargs -I {} sh -c 'cat {}; echo "\n"'
---------------------
# Copy all js files from one directory to another.

find test_data -name '*.js' | xargs -I {} cp {} copy/

------------------
# Delete all files from a directory
# and ask for confirmation for each file.

find test_data -name '*.js' | xargs -p -I {} rm {}

------------------------
# Print contents of all js files in the current directory.

find . -name *.js | xargs cat

-----------------
# Convert rspec tests from "should" and "stub" to "expect" and "allow".

convert() {
    cat $1 | \
        sed 's/^\(\s\+\)\(.*\)\(\.should\)\(.*\)$/\1expect(\2\).to\4/' | \
        sed 's/^\(\s\+\)\(.*\)\(\.stub\)\(.*\)$/\1allow(\2).to receive\4/' \
        > .tmp
    mv .tmp $1
}
export -f convert

find spec -name '*_spec.rb' | xargs -I {} bash -c "convert {}"

----------------------------


In [ ]:
循环打印
Loops over all strins and print them

for one in ABC DEF: do echo $one: done

for file in *.txt; do echo $file; done


In [ ]:
#!/bin/bash

# Create /etc/resolve.conf in cygwin needed for dig/bind utils
ipconfig /all > /tmp/foo && head -$(grep -n "Ethernet adapter Local Area Connection" /tmp/foo | awk -F: '{ print $1 }') /tmp/foo | tail -$(($(wc -l /tmp/foo | awk '{ print $1 }') - $(grep -n "Ethernet adapter Local Area Connection" /tmp/foo | awk -F: '{ print $1 }'))) &> /tmp/bar && tail -$(($(wc -l /tmp/bar | awk '{ print $1 }') - $(grep -n "DNS Servers" /tmp/bar | awk -F: '{ print $1 }') + 1)) /tmp/bar | head -3 | awk '{ print "nameserver " $NF }' > /etc/resolv.conf && rm /tmp/foo && rm /tmp/bar


# Create a large text file, but unlike using truncate or dd if=/dev/(u)random of=blah the file contains only printible characters.
echo "all ur base r belong to us" >> blah && size=`ls -l blah | awk '{ print $5 }'` && while [ $size -lt 2621440000 ]; do cp blah blah.tmp && cat blah.tmp >> blah && rm blah.tmp && size=`ls -l blah | awk '{ print $5 }'`; done;

In [ ]:
poor mans tree size

du -h <dir> | grep '[0-9\.]\+G'

copy and mv

cp <fileordir>{,_copy}
mv <fileordir>{,_old}

random pass gen

strings /dev/urandom | grep -o '[[:alnum:]]' | head -n 30 | tr -d '\n'; echo

arp

arp-scan -l -I eth0 | sort -k3 > output

find largest files

find / -type f -print0| xargs -0 ls -s | sort -rn | awk '{size=$1/1024; printf("%dMb %s\n", size,$2);}' | head -5

update resolution

sudo grubby --update-kernel=ALL --args="video=hyperv_fb:2560x1440"
© 2017 GitHub, Inc.

In [ ]:
ls -al . | awk '{print $9}' | echo $(basename $(cat -)) | tr -d '\n' | pbcopy


tail -f /var/log/my-log.log \
| awk '{ print strftime ("%Y-%m-%d %H:%M:%S "), $0, fflush();}'

# Prints out source and port of rejected (iptables) connection packets
cat /var/log/syslog | grep SYN | sed -n 's/.*DST=\([0-9\.]*\).*DPT=\([0-9]*\).*/\1 \2/p' 

# Prints unique lists of CIDR for rejected firewall ports (good for whitelisting)
cat /var/log/syslog | grep SYN | sed -n 's/.*DST=\([0-9\.]*\).*DPT=\([0-9]*\).*/\1 \2/p' | sort | uniq | ( while read ip port; do cidr=`whois $ip | grep CIDR`; name=`whois $ip|grep NetName`; echo $cidr $port $name; done ) | sort | uniq





In [ ]:
https://github.com/gaursagar/bash-one-liners

In [ ]:
  ############# removing soft line wrap from fasta #############
$ awk '!/^>/ { printf "%s", $0; n="" } /^>/ { print n $0; n = "" } END { printf "%s", n }' in.fa > out.fa 

  ############# splitting fastq to fasta with sed #############
$ sed -n '1~4s/^@/>/p;2~4p' in.fq > out.fa


  ############# replacing ^M carriage returns that excel likes to put in with newline characters #############
$ tr "\r" "\n" < paired_dists.csv > paired_dists_fixed.csv


  ############# quick de-interleave interleaved fastq files #############
$ paste - - - - - - - - < interleaved.fastq | tee >(cut -f 1-4 | tr '\t' '' > reads1.fastq) | cut -f 5-8 | tr '\t' '' > reads2.fastq


  ############# replacing newline characters with mac bash's finicky sed #############
$ sed -e ':a' -e 'N' -e '$!ba' -e 's///g' in.txt > out.txt


  ############# example to download gene nucleotide sequences from ncbi by gene id #############
$ perl -e 'use LWP::Simple;getstore("http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&retmode=text&id=".join(",",qw(6701965 6701969 6702094 6702105 6702160)),"seqs.fasta");'


  ############# count number of bases in a fasta file #############
$ grep -v ">" file.fa | wc | awk '{print $3-$1}'


  ############# replace full word only with sed on mac (enclose string with "[[:>:]]...[[:<:]]") #############
$ sed "s/[[:<:]]1[[:>:]]/A/" test.txt
  # this example replaces 1 with A only when 1 is full word, analogous to -w flag in most bash shells outside of mac


  ############# add '>' to every other line #############
$ sed 's/^/>/;n' test.fa

  ############# interleave two files line-by-line #############
$ paste -d'\n' temp_headers temp_seqs > new_fasta.fa

In [ ]:
Emptying a file

> file.txt
It is shorter than the commonly used:

echo '' > file.txt
Filesize in bytes

Instead of trying to parse ls -l output, it's easier to use stat:

stat -c %s file.txt

Prepend all lines with a string

Prepend hello to all lines:

sed 's/^/hello /'
Example:

$ echo -e 'one\ntwo\nthree' | sed 's/^/hello /'
hello one
hello two
hello three
Output only N-th line

Output only tenth line:

sed -n 10p
Output only lines M through N

Output only lines 10, 11, …, 20:

sed -n 10,20p
Remove N-th line from output

Output all lines but the tenth:

sed 10d
Output even/odd/N-th lines only

Even lines only:

sed -n '2~2p'
Odd lines only:

sed -n '1~2p'
Generic form:

sed -n 'A~Bp'
This starts with the A-th line and then outputs every B-th line.

Output all lines between two matches

Output all lines from the line containing A through the line containing B. The two boundary lines are included:

awk '/A/,/B/'
Example:

$ echo -e 'one\ntwo\nthree\nfour\nfive\nsix' | awk /two/,/five/
two
three
four
five
Output unique lines without sorting

awk '!x[$0]++'
(not suitable for large files)

Example:

$ echo -e 'three\ntwo\ntwo\none\nthree\nthree\none' | awk '!x[$0]++'
three
two
one
Repeat a string N times

echo -e $_{1..N}'\bSTRING'
(where N is the count and STRING is the word to repeat)

Without \b the words will be separated by spaces:

echo $_{1..N}'STRING'
Example: Repeat the word Hello 6 times:

$ echo -e $_{1..6}'\bHello'
HelloHelloHelloHelloHelloHello
Bash variable manipulation

Search and replace

Replace the first dot with an underscore:

$ var=a.b.c.d; echo ${var/./_}
a_b.c.d
Replace all dots with underscores:

$ var=a.b.c.d; echo ${var//./_}
a_b_c_d
Search and remove

Remove the first dot:

$ var=a.b.c.d; echo ${var/.}
ab.c.d
Remove all dots:

$ var=a.b.c.d; echo ${var//.}
abcd
Conditional replace

Replace the first character with an X if it is an a/b:

$ var=a.b.c.d; echo ${var/#a/X}
X.b.c.d

$ var=a.b.c.d; echo ${var/#b/X}
a.b.c.d
Replace the last character with an X if it is an d/e:

$ var=a.b.c.d; echo ${var/%d/X}
a.b.c.X

$ var=a.b.c.d; echo ${var/%e/X}
a.b.c.d
Strip the beginning/end according to a pattern

Remove everything after the last dot (ungreedy pattern):

$ var=a.b.c.d; echo ${var%.*}
a.b.c
Remove everything after the first dot (greedy pattern):

$ var=a.b.c.d; echo ${var%%.*}
a
Remove everything before the first dot (ungreedy pattern):

$ var=a.b.c.d; echo ${var#*.}
b.c.d
Remove everything before the last dot (greedy pattern):

$ var=a.b.c.d; echo ${var##*.}
d
Convert to lowercase/uppercase

Convert the first character to uppercase/lowercase:

$ var=abcd; echo ${var^}
Abcd

$ var=ABCD; echo ${var,}
aBCD
Convert all characters to uppercase/lowercase:

$ var=abcd; echo ${var^^}
ABCD

$ var=ABCD; echo ${var,,}
abcd
Substring

All but the first 2 characters:

$ var=abcd; echo ${var:2}
cd
3 characters, starting from the 2nd:

$ var=abcdef; echo ${var:2:3}
cde
The last 2 characters:

$ var=abcdef; echo ${var: -2}
ef
(The space before the minus/dash is necessary since :- means: use a default value if the variable is not set.)

String length

$ var=abcdef; echo ${#var}
6
Other utilities

Pick a random line

shuf -n 1
shuf shuffles all lines, -n 1 outputs only the first line.

Output lines in reverse order

Type cat in reverse:

tac
Example:

$ echo -e 'one\ntwo\nthree' | tac
three
two
one
Diff two unsorted files without creating temporary files

diff <(sort file1.txt) <(sort file2.txt)

In [ ]:
# Sort by column
	sort -n -k 1

# Cut out columns 1, 2, 3, 4, and 5
	cut -f1,2,3,4,5 filename 
 
# Sort, Uniq, Count by column 3
	more filename | sort -s -n -k 3 | uniq -f 3 -c 

# Differences between two columns in 2 files
	awk 'NR==FNR{c[$4]++;next};c[$4] == 0'	
	awk -F, 'FILENAME != ARGV[ARGC-1] {keys[$6]; next} !($6 in keys)'
	awk 'NR == FNR { a[$1,$2] = $0; next }  { delete a[$1, $2] } END { for (i in a) print a[i] }'

In [ ]:
#Count lines in multiple files and display lines per file
  grep -c filenames

#Mutation spectrum (base pair change) of SNVs in a BED file
	cut -f4,5 filename | sort | uniq -c	

#Find difference between 2 files
	awk 'FNR==NR{f[$1];next}(!($1 in f)) ' file1 file2

#Find pattern and display n lines above pattern (n=10)
	grep "pattern" filename -B 10 

#Find pattern and display n lines below pattern (n=10)
	grep "pattern" filename -A 10 

#Combine files
	cat file1 file2 file3 > newfile

#Replace ^M with line break in VI text editor
	%s/^M/\r/g

#Replace space in the beginning of line VI text editor
	%s/^ \+//

#Compare file1 and file2; suppress lines unique to FILE1 
	comm -1 file1 file2 > file3

#Compare file1 and file2, suppress lines unique to FILE2
	comm -2 file1 file2 > file3

#Compare file1 and file2, suppress lines that appear in both files
	comm -3 file1 file2 > file3

#Extract last field per line
	awk '{ print $NF }' filename
	
#Mutation spectrum (base Pair change) of SNVs in a BED file
	cut -f4,5 filename | sort | uniq -c

#Extract the Nth string in a line
	cut -c Nth	
	
#Unzip multiple tar files
	for file in *.tar.gz; do tar -zxf $file; done

#Extract the 8th line of single file
	sed '8q;d' filename	

#Extract the 8th line of multiple files	
	awk 'FNR == 8' *filenames
	
#Find 2 patterns in the same file
	cat filename | grep -e "pattern1"  -e "pattern2"
	
#Append output to beginning of file
	cat <(command) filename >tmp && mv tmp filename


In [ ]:
#count number of sequences in a fasta file
	grep -c "^>" file.fa

#Sequence length distribution in a Fastq File
	awk 'NR%4 == 2 {lengths[length($0)]++} END {for (l in lengths) {print l, lengths[l]}}' file.fastq
	cat file.fastq | awk '{if(NR%4==2) print length($1)}' | sort -n | uniq -c 

#number of reads in a bam file
	samtools idxstats in.bam | awk '{s+=$3+$4} END {print s}'

#number of mapped reads in a bam file
	samtools idxstats in.bam | awk '{s+=$3} END {print s}'

#convert fasta file to BLAST database
	formatdb -p F -i fasta-file -n name-of-BLASTdb

#convert SAM file to BAM file 
	samtools view in.bam > out.sam 

#convert BAM file to SAM file 
	samtools view in.sam > out.sam 
	
#Extract fasta sequences with IDs in a seperate file
	perl -ne 'if(/^>(\S+)/){$c=$i{$1}}$c?print:chomp;$i{$_}=1 if @ARGV' ids.file fasta.file
	
#Find "." replace with nothing
	sed  "s/\.//g" file.in >> file.out 

#Find space replace with pipe "|"
	sed 's/[ \t]/|/g' file.in >> file.out 

#Delete lines containing obsolete
	sed '/obsolete/d' file.in >> file.out 

#Clean fasta file downloaded from NCBI to retain only species names 
#Open file in favortie text editor
#Search with the following string and replace with nothing
	gi\|[0-9]+\|[A-Z]+\|.+\|.+\[ 
	
#Remove empty headers with no sequences in fasta file
	awk 'BEGIN {RS = ">" ; FS = "\n" ; ORS = ""} $2 {print ">"$0}' input.fasta > output.fasta


In [ ]:
#Total number of variants in vcf file
	grep -v "#" filename | wc -l
		 
#Total number of variants in vcf file that passed filtering
	awk '{if($7=="PASS") print}' filename | wc -l

#Number of unique SNVs across all sample in a Monovar output file
	cut -f20 MonoVar_output_file.vcf | sed 1,20d | sed 's/[^1,2]//g'  | awk '{ print length }' | sort | wc -l
	
#Number of unique SNVs in at least 2 samples within a Monovar output file
	cut -f20 MonoVar_output_file.vcf   | sed 1,20d | sed 's/[^1,2]//g'  | awk '{ print length }' | sort | awk '$1> 1' | wc -l
	
#Number of unique SNVs in at least 3 samples within a Monovar output file
	cut -f20 MonoVar_output_file.vcf   | sed 1,20d | sed 's/[^1,2,3]//g'  | awk '{ print length }' | sort | awk '$1> 2' | wc -l


In [6]:
%%ruby
puts 123

123


In [ ]:
在一个变量中把多个空白合并成一个空白，用 Perl 是这么一句话：
$text =~ s/(\s)+/( )/g;

ls -l | awk '$1=="-rw-r--r--" {x += $5} END {print "Total bytes = " x}'
# sum the file sizes in bytes of all files in a directory

# Shell one liners, using awk or something else!

# Give a 2 column, tab separated list of read no. and read length
zcat whatever.fastq.gz | paste - - - - | awk '{print NR " " (length($3))}'


# Count total reads in fastq file
zcat whatever.fastq.gz | wc -l | awk '{print $1/4}' 

# Change extension of multiple files at once.
# In below example, the extension changes from *.scafSeq to *.fa
for f in *.scafSeq; do mv "$f" "$(basename "$f" .scafSeq).fa"; done


# get A T G C counts for all sequences from a multi fasta file
echo -e "seq_id\tA\tU\tG\tC"; while read line; do echo $line | grep ">" | sed 's/>//g'; for i in A U G C;do echo $line | grep -v ">" | grep -o $i | wc -l | grep -v "^0"; done; done < test.fa | paste - - - - -

#counting number of sequences in a fasta file:
grep -c "^>" file.fa

#add something to end of all header lines:
sed 's/>.*/&WHATEVERYOUWANT/' file.fa > outfile.fa

#clean up a fasta file so only first column of the header is outputted:
awk '{print $1}' file.fa > output.fa

Print only line number 101 in file:
awk 'NR==101' file

Print lines 101-202 in file:
awk 'NR==101, NR==202' file

Print every 4th line in a file:
awk '0 == NR % 4' file

Print header with column/field numbers in a tab-separated file:
awk -F "\t" '{ for (f=1; f<=NF; f++) print f":"$f; exit }' file

Rename multiple files (/bin/sh executes):
ls * | awk '{ print "mv "$1" "$1 }' | sed 's/to_replace/replace_with/2' | /bin/sh

Remove duplicate, nonconsecutive lines (no need to sort first):
awk '!a[$0]++' file

Put every other line next to the one above, separated by tab:
awk 'ORS=NR%2 ? "\t" : "\n"' file

Print section of file between two regexes:
awk '/regex1/, /regex2/' file

Print average of third column:
awk '{ sum += $3 } END { if (NR > 0) print sum / NR }' file

Sort on 3rd column, then print fields 2 through 5 of a tabular file, and align output:
sort -nk 3 file | cut -f 2-5 | column -t

Print number of files present in each subdirectory of the current directory:
find . -maxdepth 1 -type d -exec bash -c "echo -ne '{} '; ls '{}' | wc -l" \;

Check if the same file exists in all directories:
for d in ./bin*; do [[ -f ./$d/RAxML_bootstrap-out ]] && echo "File exist in dir $d" || echo "File does not exist in dir $d"; done

Find directories that are missing files with the word masked in their name:
find base_dir -type d '!' -exec sh -c 'ls -1 "{}" | grep -q "*masked*"' ';' -print




In [ ]:
sed

 # spasi dobel untuk sebuah berkas
 sed G

 # spasi dobel sebuah berkas yang sudah memiliki baris-baris kosong di
 # dalamnya. Berkas keluaran nanti berisi tidak lebih dari satu baris 
 # kosong di antara baris-baris teks
 sed '/^$/d;G'


 # spasi tripel untuk sebuah berkas
 sed 'G;G'

 # undo spasi-dobel (anggap baris-baris bernomor genap selalu kosong)
 sed 'n;d'
 
 # sisipkan satu baris kosong di atas setiap baris yang cocok dengan "regex"
 sed '/regex/{x;p;x;}'

 # sisipkan sebuah baris kosong di bawah setiap baris yang cocok dengan "regex"
 sed '/regex/G'

 # insert a blank line above and below every line which matches "regex"
 sed '/regex/{x;p;x;G;}'

PENOMORAN:

 # number each line of a file (simple left alignment). Using a tab (see
 # note on '\t' at end of file) instead of space will preserve margins.
 sed = filename | sed 'N;s/\n/\t/'

 # number each line of a file (number on left, right-aligned)
 sed = filename | sed 'N; s/^/     /; s/ *\(.\{6,\}\)\n/\1  /'

 # number each line of file, but only print numbers if line is not blank
 sed '/./=' filename | sed '/./N; s/\n/ /'

 # hitung baris (meniru "wc -l)
 sed -n '$='

KONVERSI TEKS DAN SUBSTITUSI:

 # DI LINGKUNGAN UNIX: convert DOS newlines (CR/LF) to Unix format.
 sed 's/.$//'               # assumes that all lines end with CR/LF
 sed 's/^M$//'              # in bash/tcsh, press Ctrl-V then Ctrl-M
 sed 's/\x0D$//'            # works on ssed, gsed 3.02.80 or higher

 # DI LINGKUNGAN UNIX: convert Unix newlines (LF) to DOS format.
 sed "s/$/`echo -e \\\r`/"            # command line under ksh
 sed 's/$'"/`echo \\\r`/"             # command line under bash
 sed "s/$/`echo \\\r`/"               # command line under zsh
 sed 's/$/\r/'                        # gsed 3.02.80 or higher

 # DI LINGKUNGAN DOS: convert Unix newlines (LF) to DOS format.
 sed "s/$//"                          # method 1
 sed -n p                             # method 2

 # DI LINGKUNGAN DOS: convert DOS newlines (CR/LF) to Unix format.
 # Can only be done with UnxUtils sed, version 4.0.7 or higher. The
 # UnxUtils version can be identified by the custom "--text" switch
 # which appears when you use the "--help" switch. Otherwise, changing
 # DOS newlines to Unix newlines cannot be done with sed in a DOS
 # environment. Use "tr" instead.
 sed "s/\r//" infile >outfile         # UnxUtils sed v4.0.7 or higher
 tr -d \r <infile >outfile            # GNU tr version 1.22 or higher

 # hapus spasi atau tab awal di depan setiap baris
 # dengan kata lain, ratakan kiri seluruh teks
 sed 's/^[ \t]*//'                    # see note on '\t' at end of file

 # delete trailing whitespace (spaces, tabs) from end of each line
 sed 's/[ \t]*$//'                    # see note on '\t' at end of file

 # delete BOTH leading and trailing whitespace from each line
 sed 's/^[ \t]*//;s/[ \t]*$//'

 # sisipkan 5 spasi kosong pada awal setiap baris (membuat page offset)
 sed 's/^/     /'

 # ratakan semua teks ke kanan dengan lebar 79 kolom
 sed -e :a -e 's/^.\{1,78\}$/ &/;ta'  # set at 78 plus 1 space

 # center all text in the middle of 79-column width. In method 1,
 # spaces at the beginning of the line are significant, and trailing
 # spaces are appended at the end of the line. In method 2, spaces at
 # the beginning of the line are discarded in centering the line, and
 # no trailing spaces appear at the end of lines.
 sed  -e :a -e 's/^.\{1,77\}$/ & /;ta'                     # method 1
 sed  -e :a -e 's/^.\{1,77\}$/ &/;ta' -e 's/\( *\)\1/\1/'  # method 2

 # substitusi (find and replace) "foo" dengan "bar" pada setiap baris
 sed 's/foo/bar/'             # replaces only 1st instance in a line
 sed 's/foo/bar/4'            # replaces only 4th instance in a line
 sed 's/foo/bar/g'            # replaces ALL instances in a line
 sed 's/\(.*\)foo\(.*foo\)/\1bar\2/' # replace the next-to-last case
 sed 's/\(.*\)foo/\1bar/'            # replace only the last case

 # substitusi "foo" dengan "bar" HANYA untuk baris yang berisi "baz"
 sed '/baz/s/foo/bar/g'

 # substitusi "foo" dengan "bar" KECUALI untuk baris yang berisi "baz"
 sed '/baz/!s/foo/bar/g'

 # ubah "scarlet" atau "ruby" atau "puce" menjadi "red"
 sed 's/scarlet/red/g;s/ruby/red/g;s/puce/red/g'   # most seds
 gsed 's/scarlet\|ruby\|puce/red/g'                # GNU sed only

 # reverse order of lines (emulates "tac")
 # bug/feature in HHsed v1.5 causes blank lines to be deleted
 sed '1!G;h;$!d'               # method 1
 sed -n '1!G;h;$p'             # method 2

 # balik setiap karakter pada baris sekarang (meniru "rev")
 sed '/\n/!G;s/\(.\)\(.*\n\)/&\2\1/;//D;s/.//'

 # gabung baris-baris berpasangan dua-dua (seperti "paste")
 sed '$!N;s/\n/ /'

 # jika sebuah baris berakhiran backslash, bubuhkan baris selanjutnya
 sed -e :a -e '/\\$/N; s/\\\n//; ta'

 # jika sebuah baris diawali tanda sama dengan, bubuhkan kepada baris
 # sebelumnya dan ganti "=" dengan satu spasi
 sed -e :a -e '$!N;s/\n=/ /;ta' -e 'P;D'

 # tambahkan koma ke string numerik, mengubah "1234567" jadi "1,234,567"
 gsed ':a;s/\B[0-9]\{3\}\>/,&/;ta'                     # GNU sed
 sed -e :a -e 's/\(.*[0-9]\)\([0-9]\{3\}\)/\1,\2/;ta'  # other seds

 # tambahkan koma ke nomor dengan titik desimal dan tanda minus (GNU sed)
 gsed -r ':a;s/(^|[^0-9.])([0-9]+)([0-9]{3})/\1\2,\3/g;ta'

 # tambah baris kosong setiap lima baris (setelah baris 5, 10, 15, dst)
 gsed '0~5G'                  # GNU sed saja
 sed 'n;n;n;n;G;'             # sed lainnya
 
PENCETAKAN SELEKTIF DARI BARIS TERTENTU:

 # cetak 10 baris pertama berkas (meniru perilaku "head")
 sed 10q

 # cetak baris pertama berkas (meniru "head -1")
 sed q

 # cetak 10 baris terakhir berkas (meniru "tail")
 sed -e :a -e '$q;N;11,$D;ba'

 # cetak 2 baris terkahir sebuah berkas (meniru "tail -2")
 sed '$!N;$!D'

 # cetak baris terakhir sebuah berkas (meniru "tail -1")
 sed '$!d'                    # method 1
 sed -n '$p'                  # method 2

 __# cetak baris tepat di bawah sampai ke akhir berkas__
 sed -e '$!{h;d;}' -e x              # for 1-line files, print blank line
 sed -e '1{$q;}' -e '$!{h;d;}' -e x  # for 1-line files, print the line
 sed -e '1{$d;}' -e '$!{h;d;}' -e x  # for 1-line files, print nothing

 # cetak hanya baris yang sesuai regex (meniru "grep")
 sed -n '/regexp/p'           # method 1
 sed '/regexp/!d'             # method 2

 # cetak hanya baris yang TIDAK sesuai regex (meniru "grep -v")
 sed -n '/regexp/!p'          # method 1, corresponds to above
 sed '/regexp/d'              # method 2, simpler syntax

 # cetak baris tepat sebelum regex, tetapi bukan yang berisi regex
 sed -n '/regexp/{g;1!p;};h'

 # cetak baris tepat setelah regex, tetapi bukan yang berisi regex
 sed -n '/regexp/{n;p;}'

 # cetak 1 baris dari konteks sebelum dan sesudah regex, dengan nomor baris
 # menunjukkan di mana regex terjadi (seperti "gre[p -A1 -B1")
 sed -n -e '/regexp/{=;x;1!p;g;$!N;p;D;}' -e h

 # grep untuk AAA dan BBB dan CCC (urutan terserah)
 sed '/AAA/!d; /BBB/!d; /CCC/!d'

 # grep untuk AAA dan BBB dan CCC (sesuai urutan)
 sed '/AAA.*BBB.*CCC/!d'

 # grep untuk AAA atau BBB atau CCC (meniru "egrep")
 sed -e '/AAA/b' -e '/BBB/b' -e '/CCC/b' -e d    # most seds
 gsed '/AAA\|BBB\|CCC/!d'                        # GNU sed only

 # print paragraph if it contains AAA (blank lines separate paragraphs)
 # HHsed v1.5 must insert a 'G;' after 'x;' in the next 3 scripts below
 sed -e '/./{H;$!d;}' -e 'x;/AAA/!d;'

 # print paragraph if it contains AAA and BBB and CCC (in any order)
 sed -e '/./{H;$!d;}' -e 'x;/AAA/!d;/BBB/!d;/CCC/!d'

 # print paragraph if it contains AAA or BBB or CCC
 sed -e '/./{H;$!d;}' -e 'x;/AAA/b' -e '/BBB/b' -e '/CCC/b' -e d
 gsed '/./{H;$!d;};x;/AAA\|BBB\|CCC/b;d'         # GNU sed only

 # print only lines of 65 characters or longer
 sed -n '/^.\{65\}/p'

 # cetak hanya baris yang berisi kurang dari 65 karakter
 sed -n '/^.\{65\}/!p'        # method 1, corresponds to above
 sed '/^.\{65\}/d'            # method 2, simpler syntax

 # cetak bagian berkas dari regex hingga akhir berkas
 sed -n '/regexp/,$p'

 # cetak bagian berkas berdasarkan nomor baris (baris 8-12, inklusif)
 sed -n '8,12p'               # method 1
 sed '8,12!d'                 # method 2

 # cetak baris nomor 52
 sed -n '52p'                 # method 1
 sed '52!d'                   # method 2
 sed '52q;d'                  # method 3, efficient on large files

 # mulai dari baris tiga, cetak setiap baris ke-7
 gsed -n '3~7p'               # GNU sed only
 sed -n '3,${p;n;n;n;n;n;n;}' # other seds

 # print section of file between two regular expressions (inclusive)
 # cetak bagian berkas antara dua regex (inklusif)
 sed -n '/Iowa/,/Montana/p'             # case sensitive

PENGHAPUSAN SELEKTIF BARIS TERTENTU:

 # cetak semua berkas KECUALI bagian di antara 2 regex
 sed '/Iowa/,/Montana/d'

 # hapus duplikat, baris berderet dari sebuah berkas (meniru "uniq")
 # Baris pertama dalam sebuah set duplikat dibiarkan, sisanya dihapus
 sed '$!N; /^\(.*\)\n\1$/!P; D'

 # delete duplicate, nonconsecutive lines from a file. Beware not to
 # overflow the buffer size of the hold space, or else use GNU sed.
 sed -n 'G; s/\n/&&/; /^\([ -~]*\n\).*\n\1/d; s/\n//; h; P'

 # hapus semua baris kecuali baris-baris duplikat (meniru "uniq")
 sed '$!N; s/^\(.*\)\n\1$/\1/; t; D'

 # hapus 10 baris pertama sebuah berkas
 sed '1,10d'

 # hapus baris terakhir sebuah berkas
 sed '$d'

 # hapus 2 baris terakhir sebuah berkas
 sed 'N;$!P;$!D;$d'

 # hapus 10 baris terakhir dari sebuah berkas
 sed -e :a -e '$d;N;2,10ba' -e 'P;D'   # method 1
 sed -n -e :a -e '1,10!{P;N;D;};N;ba'  # method 2

 # hapus setiap baris ke-8
 gsed '0~8d'                           # GNU sed only
 sed 'n;n;n;n;n;n;n;d;'                # other seds

 # hapus baris-baris yang cocok dengan pola
 sed '/pattern/d'

 # hapus SEMUA baris kosong dari sebuah berkas (sama dengan "grep '.' ")
 sed '/^$/d'                           # method 1
 sed '/./!d'                           # method 2

 # delete all CONSECUTIVE blank lines from file except the first; also
 # deletes all blank lines from top and end of file (emulates "cat -s")
 sed '/./,/^$/!d'          # method 1, allows 0 blanks at top, 1 at EOF
 sed '/^$/N;/\n$/D'        # method 2, allows 1 blank at top, 0 at EOF

 # delete all CONSECUTIVE blank lines from file except the first 2:
 sed '/^$/N;/\n$/N;//D'

 # delete all leading blank lines at top of file
 sed '/./,$!d'

 # delete all trailing blank lines at end of file
 sed -e :a -e '/^\n*$/{$d;N;ba' -e '}'  # works on all seds
 sed -e :a -e '/^\n*$/N;/\n$/ba'        # ditto, except for gsed 3.02.*

 # hapus baris terakhir dari setiap paragraf
 sed -n '/^$/{p;h;};/./{x;/./p;}'

PENERAPAN ISTIMEWA:

 # remove nroff overstrikes (char, backspace) from man pages. The 'echo'
 # command may need an -e switch if you use Unix System V or bash shell.
 sed "s/.`echo \\\b`//g"    # double quotes required for Unix environment
 sed 's/.^H//g'             # in bash/tcsh, press Ctrl-V and then Ctrl-H
 sed 's/.\x08//g'           # hex expression for sed 1.5, GNU sed, ssed

 # get Usenet/e-mail message header
 sed '/^$/q'                # deletes everything after first blank line

 # get Usenet/e-mail message body
 sed '1,/^$/d'              # deletes everything up to first blank line

 # get Subject header, but remove initial "Subject: " portion
 sed '/^Subject: */!d; s///;q'

 # get return address header
 sed '/^Reply-To:/q; /^From:/h; /./d;g;q'

 # parse out the address proper. Pulls out the e-mail address by itself
 # from the 1-line return address header (see preceding script)
 sed 's/ *(.*)//; s/>.*//; s/.*[:<] *//'

 # add a leading angle bracket and space to each line (quote a message)
 sed 's/^/> /'

 # delete leading angle bracket & space from each line (unquote a message)
 sed 's/^> //'

 # remove most HTML tags (accommodates multiple-line tags)
 sed -e :a -e 's/<[^>]*>//g;/</N;//ba'

 # extract multi-part uuencoded binaries, removing extraneous header
 # info, so that only the uuencoded portion remains. Files passed to
 # sed must be passed in the proper order. Version 1 can be entered
 # from the command line; version 2 can be made into an executable
 # Unix shell script. (Modified from a script by Rahul Dhesi.)
 sed '/^end/,/^begin/d' file1 file2 ... fileX | uudecode   # vers. 1
 sed '/^end/,/^begin/d' "$@" | uudecode                    # vers. 2

 # sort paragraphs of file alphabetically. Paragraphs are separated by blank
 # lines. GNU sed uses \v for vertical tab, or any unique char will do.
 sed '/./{H;d;};x;s/\n/={NL}=/g' file | sort | sed '1s/={NL}=//;s/={NL}=/\n/g'
 gsed '/./{H;d};x;y/\n/\v/' file | sort | sed '1s/\v//;y/\v/\n/'

 # zip up each .TXT file individually, deleting the source file and
 # setting the name of each .ZIP file to the basename of the .TXT file
 # (under DOS: the "dir /b" switch returns bare filenames in all caps).
 echo @echo off >zipup.bat
 dir /b *.txt | sed "s/^\(.*\)\.TXT/pkzip -mo \1 \1.TXT/" >>zipup.bat

TYPICAL USE: Sed takes one or more editing commands and applies all of
them, in sequence, to each line of input. After all the commands have
been applied to the first input line, that line is output and a second
input line is taken for processing, and the cycle repeats. The
preceding examples assume that input comes from the standard input
device (i.e, the console, normally this will be piped input). One or
more filenames can be appended to the command line if the input does
not come from stdin. Output is sent to stdout (the screen). Thus:

 cat filename | sed '10q'        # uses piped input
 sed '10q' filename              # same effect, avoids a useless "cat"
 sed '10q' filename > newfile    # redirects output to disk

For additional syntax instructions, including the way to apply editing
commands from a disk file instead of the command line, consult "sed &
awk, 2nd Edition," by Dale Dougherty and Arnold Robbins (O'Reilly,
1997; http://www.ora.com), "UNIX Text Processing," by Dale Dougherty
and Tim O'Reilly (Hayden Books, 1987) or the tutorials by Mike Arst
distributed in U-SEDIT2.ZIP (many sites). To fully exploit the power
of sed, one must understand "regular expressions." For this, see
"Mastering Regular Expressions" by Jeffrey Friedl (O'Reilly, 1997).
The manual ("man") pages on Unix systems may be helpful (try "man
sed", "man regexp", or the subsection on regular expressions in "man
ed"), but man pages are notoriously difficult. They are not written to
teach sed use or regexps to first-time users, but as a reference text
for those already acquainted with these tools.

QUOTING SYNTAX: The preceding examples use single quotes ('...')
instead of double quotes ("...") to enclose editing commands, since
sed is typically used on a Unix platform. Single quotes prevent the
Unix shell from intrepreting the dollar sign ($) and backquotes
(`...`), which are expanded by the shell if they are enclosed in
double quotes. Users of the "csh" shell and derivatives will also need
to quote the exclamation mark (!) with the backslash (i.e., \!) to
properly run the examples listed above, even within single quotes.
Versions of sed written for DOS invariably require double quotes
("...") instead of single quotes to enclose editing commands.

USE OF '\t' IN SED SCRIPTS: For clarity in documentation, we have used
the expression '\t' to indicate a tab character (0x09) in the scripts.
However, most versions of sed do not recognize the '\t' abbreviation,
so when typing these scripts from the command line, you should press
the TAB key instead. '\t' is supported as a regular expression
metacharacter in awk, perl, and HHsed, sedmod, and GNU sed v3.02.80.

VERSIONS OF SED: Versions of sed do differ, and some slight syntax
variation is to be expected. In particular, most do not support the
use of labels (:name) or branch instructions (b,t) within editing
commands, except at the end of those commands. We have used the syntax
which will be portable to most users of sed, even though the popular
GNU versions of sed allow a more succinct syntax. When the reader sees
a fairly long command such as this:

   sed -e '/AAA/b' -e '/BBB/b' -e '/CCC/b' -e d

it is heartening to know that GNU sed will let you reduce it to:

   sed '/AAA/b;/BBB/b;/CCC/b;d'      # or even
   sed '/AAA\|BBB\|CCC/b;d'

In addition, remember that while many versions of sed accept a command
like "/one/ s/RE1/RE2/", some do NOT allow "/one/! s/RE1/RE2/", which
contains space before the 's'. Omit the space when typing the command.

OPTIMIZING FOR SPEED: If execution speed needs to be increased (due to
large input files or slow processors or hard disks), substitution will
be executed more quickly if the "find" expression is specified before
giving the "s/.../.../" instruction. Thus:

   sed 's/foo/bar/g' filename         # standard replace command
   sed '/foo/ s/foo/bar/g' filename   # executes more quickly
   sed '/foo/ s//bar/g' filename      # shorthand sed syntax

On line selection or deletion in which you only need to output lines
from the first part of the file, a "quit" command (q) in the script
will drastically reduce processing time for large files. Thus:

   sed -n '45,50p' filename           # print line nos. 45-50 of a file
   sed -n '51q;45,50p' filename       # same, but executes much faster

If you have any additional scripts to contribute or if you find errors
in this document, please send e-mail to the compiler. Indicate the
version of sed you used, the operating system it was compiled for, and
the nature of the problem. To qualify as a one-liner, the command line
must be 65 characters or less. Various scripts in this file have been
written or contributed by:

 Al Aab                   # founder of "seders" list
 Edgar Allen              # various
 Yiorgos Adamopoulos      # various
 Dale Dougherty           # author of "sed & awk"
 Carlos Duarte            # author of "do it with sed"
 Eric Pement              # author of this document
 Ken Pizzini              # author of GNU sed v3.02
 S.G. Ravenhall           # great de-html script
 Greg Ubben               # many contributions & much help
-------------------------------------------------------------------------
perl -ne 'if ( /^>/ ) {print "\n";print;} else {chomp; print;}' $1

                                                         
                                                         


In [ ]:
Print only line number 101 in file:
awk 'NR==101' file
Print lines 101-202 in file:
awk 'NR==101, NR==202' file
Print every 4th line in a file:
awk '0 == NR % 4' file
Print header with column/field numbers in a tab-separated file:
awk -F "\t" '{ for (f=1; f<=NF; f++) print f":"$f; exit }' file
Rename multiple files (/bin/sh executes):
ls * | awk '{ print "mv "$1" "$1 }' | sed 's/to_replace/replace_with/2' | /bin/sh
Remove duplicate, nonconsecutive lines (no need to sort first):
awk '!a[$0]++' file
Put every other line next to the one above, separated by tab:
awk 'ORS=NR%2 ? "\t" : "\n"' file
Print section of file between two regexes:
awk '/regex1/, /regex2/' file
Print average of third column:
awk '{ sum += $3 } END { if (NR > 0) print sum / NR }' file
Sort on 3rd column, then print fields 2 through 5 of a tabular file, and align output:
sort -nk 3 file | cut -f 2-5 | column -t
Print number of files present in each subdirectory of the current directory:
find . -maxdepth 1 -type d -exec bash -c "echo -ne '{} '; ls '{}' | wc -l" \;
Check if the same file exists in all directories:
for d in ./bin*; do [[ -f ./$d/RAxML_bootstrap-out ]] && echo "File exist in dir $d" || echo "File does not exist in dir $d"; done
Find directories that are missing files with the word masked in their name:
find base_dir -type d '!' -exec sh -c 'ls -1 "{}" | grep -q "*masked*"' ';' -print

---------
